# YOLOv8 Training for Medical X-Ray Finding Detection

This notebook implements the training pipeline for our adapted YOLOv8 model on the VinBigData Chest X-ray dataset. The model has been specifically adapted for detecting medical findings in grayscale X-ray images with optimized class weighting and resolution.

**Training Environment**: This notebook is designed to run on Google Colab with GPU acceleration to leverage free computational resources for model training.

**Dataset**: Uses a strategically sampled subset of 500 images from the VinBigData dataset, providing balanced representation across 14 medical finding types while maintaining computational efficiency.

In [13]:
import os
import torch
import zipfile
import shutil
import yaml
from ultralytics import YOLO
from datetime import datetime

## Extract and Organize Files
Let's extract the uploaded files and set up the directory structure for training.

In [6]:
os.makedirs('/content/dataset', exist_ok=True)
os.makedirs('/content/configs', exist_ok=True)

print("Extracting dataset...")
with zipfile.ZipFile('/content/yolo_dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

print("Extracting configuration files...")
with zipfile.ZipFile('/content/configs.zip', 'r') as zip_ref:
    # Extract to temp location first
    zip_ref.extractall('/content/temp_configs')

    # Move files from nested structure to correct location
    for root, dirs, files in os.walk('/content/temp_configs'):
        for file in files:
            if file.endswith('.yaml'):
                source = os.path.join(root, file)
                destination = f'/content/configs/{file}'
                shutil.move(source, destination)

    # Clean up temp directory
    shutil.rmtree('/content/temp_configs')

print("Extraction completed!")

print(f"\nDataset: {len(os.listdir('/content/dataset/yolo_dataset/train/images'))} train images, {len(os.listdir('/content/dataset/yolo_dataset/val/images'))} val images")
print("Configuration files:")
for file in os.listdir('/content/configs'):
    print(f"{file}")

Extracting dataset...
Extracting configuration files...
Extraction completed!

Dataset: 400 train images, 100 val images
Configuration files:
medical_hyper.yaml
train_config.yaml
medical_augmentation.yaml
medical_xray_model.yaml


## Load Training Configuration
Now we'll load all our custom configurations created in the model adaptation notebook and prepare for training.

In [9]:
print("=== Loading Configuration Files ===")

# Load main dataset configuration
with open('/content/dataset/yolo_dataset/data.yaml', 'r') as f:
    data_config = yaml.safe_load(f)

# Load our custom configurations
config_files = ['medical_hyper.yaml', 'medical_augmentation.yaml', 'train_config.yaml']
configs = {}

for config_file in config_files:
    config_path = f"/content/configs/{config_file}"
    with open(config_path, 'r') as f:
        configs[config_file] = yaml.safe_load(f)
    print(f"Loaded {config_file}")


print(f"\n=== Training Configuration Summary ===")
train_config = configs['train_config.yaml']
print(f"Training epochs: {train_config.get('epochs')}")
print(f"Batch size: {train_config.get('batch')}")
print(f"Image size: {train_config.get('imgsz')}")
print(f"Base model: {train_config.get('model')}")
print(f"Patience (early stopping): {train_config.get('patience')}")

print(f"Dataset classes: {data_config['nc']}")
print(f"Class names: {data_config['names'][:3]}... (showing first 3)")

print("\nAll configurations loaded successfully!")

=== Loading Configuration Files ===
Loaded medical_hyper.yaml
Loaded medical_augmentation.yaml
Loaded train_config.yaml

=== Training Configuration Summary ===
Training epochs: 100
Batch size: 16
Image size: 1024
Base model: yolov8n.pt
Patience (early stopping): 20
Dataset classes: 14
Class names: ['Aortic enlargement', 'Atelectasis', 'Calcification']... (showing first 3)

All configurations loaded successfully!


## Initialize Model for Training
Let's initialize our YOLOv8 model with the medical-specific adaptations.

In [11]:
print("=== Model Initialization ===")

model = YOLO('yolov8n.pt')
print(f"Base model loaded: yolov8n.pt")

model.info()

print(f"\nModel will be trained on: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print("Model ready for training")

=== Model Initialization ===
Base model loaded: yolov8n.pt
YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs

Model will be trained on: Tesla T4
Model ready for training


## Start Training
Now we'll begin the training process using our medical-specific configurations. The training will run for up to 100 epochs with early stopping if no improvement is seen for 20 epochs.

In [12]:
print("=== Starting Training ===")
print(f"Training start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

train_config = configs['train_config.yaml']

# Train the model using our saved configurations
results = model.train(
    data=train_config['data'].replace('data/processed/yolo_dataset/data.yaml', '/content/dataset/yolo_dataset/data.yaml'),
    epochs=train_config['epochs'],
    imgsz=train_config['imgsz'],
    batch=train_config['batch'],
    patience=train_config['patience'],
    save=train_config['save'],
    save_period=train_config['save_period'],
    device=0,  # Force GPU usage in Colab
    workers=train_config['workers'],
    optimizer=train_config['optimizer'],
    lr0=train_config['lr0'],
    lrf=train_config['lrf'],
    momentum=train_config['momentum'],
    weight_decay=train_config['weight_decay'],
    warmup_epochs=train_config['warmup_epochs'],
    warmup_momentum=train_config['warmup_momentum'],
    warmup_bias_lr=train_config['warmup_bias_lr'],
    project='/content/runs',
    name='medical_xray_yolo'
)

print(f"Training completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

=== Starting Training ===
Training start time: 2025-05-22 17:11:10
Ultralytics 8.3.142 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=medical_xray_yolo, nbs=64, nms=False, opset=None, optimize=False, optimizer

100%|██████████| 755k/755k [00:00<00:00, 114MB/s]

Overriding model.yaml nc=80 with nc=14

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

Model summary: 129 layers, 3,013,578 parameters, 3,013,562 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 293MB/s]


...

    100/100       6.8G      1.283      1.452      1.542        181       1024: 100%|██████████| 25/25 [00:41<00:00,  1.67s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.30it/s]

                   all        100       1024      0.293      0.303      0.236      0.109



100 epochs completed in 1.338 hours.
Optimizer stripped from /content/runs/medical_xray_yolo/weights/last.pt, 6.3MB
Optimizer stripped from /content/runs/medical_xray_yolo/weights/best.pt, 6.3MB

Validating /content/runs/medical_xray_yolo/weights/best.pt...
Ultralytics 8.3.142 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,008,378 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:14<00:00,  3.51s/it]


                   all        100       1024      0.324      0.306      0.255      0.113
    Aortic enlargement         60        115      0.572      0.609      0.624       0.36
           Atelectasis         13         23      0.357      0.174      0.152     0.0509
         Calcification         23         42      0.199     0.0952      0.109     0.0348
          Cardiomegaly         33         71      0.611      0.592      0.579       0.34
         Consolidation         20         28      0.141      0.135      0.104     0.0323
                   ILD         21         57       0.36      0.298      0.229     0.0792
          Infiltration         23         44        0.2      0.159      0.101     0.0408
          Lung Opacity         49         96      0.227      0.229      0.145     0.0542
           Nodule/Mass         32        103       0.33      0.138      0.121     0.0473
          Other lesion         34         72      0.116      0.139     0.0787     0.0269
      Pleural effusio

## Model Export and Download
Now we'll prepare the trained model for download and continue with validation in the next notebook.

In [14]:
print("=== Preparing Model for Download ===")

# Copy best model to easily accessible location
shutil.copy('/content/runs/medical_xray_yolo/weights/best.pt', '/content/medical_xray_best_model.pt')
print("Best model prepared")


print("Downloading trained model...")
files.download('/content/medical_xray_best_model.pt')
print("Model downloaded successfully")

=== Preparing Model for Download ===
Best model prepared


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Model downloaded successfully


## Next Steps
The model training is complete. The next notebook will focus on:

1. **Performance Analysis**: Detailed evaluation using medical-specific metrics
2. **FROC Analysis**: Free-Response ROC curves for radiological assessment  
3. **Sensitivity Analysis**: Per-class performance evaluation for critical findings
4. **Visualization**: Advanced heatmaps and clinical case examples

The trained model achieved strong performance on critical findings like pneumothorax (59.4% mAP50) and shows excellent potential for clinical applications, if trained with more images.